In [1]:
import numpy as np
import pandas as pd
import os
import math
from datetime import datetime, timedelta
from sklearn import linear_model

In [10]:
data_directory = "../Data/Study/16/"
subdirectory = 'shifted/'
fileList = os.listdir(data_directory + subdirectory)

In [3]:
def clean_df(df):
    result = df.loc[(df['Movement'] != "start") & (df['Movement'] != "transition") & (df['EyeTrackingEnabled'] == True) & (df['EyeCalibrationValid'] == True) & (df['EyeTrackingEnabledAndValid'] == True) & (df['EyeTrackingDataValid'] == True) & (df['GazeInputSource'] == "Eyes")].copy()
    #result = result[result['Movement'] != "transition"]
    #result = result[result['EyeTrackingDataValid'] == 'TRUE']
    #result = result[result['GazeInputSource'] == 'Eyes']
    result.reset_index(inplace=True, drop=True)
    return result

In [4]:
def create_calibration_df(taskName):
    frames = []
    for file in fileList:
        if taskName in file:
            df = pd.read_csv(data_directory + subdirectory + file)
            df = clean_df(df)
            frames.append(df)
    result = pd.concat(frames)
    return result

In [5]:
def calculate_regression_coefficients(df):
    x1 = df[['GazeAngleX','GazeAngleY']]
    y1 = df['TargetAngleX']
    x2 = df[['GazeAngleX','GazeAngleY']]
    y2 = df['TargetAngleY']
    x_regr = linear_model.LinearRegression()
    x_regr.fit(x1.to_numpy(), y1.to_numpy())
    y_regr = linear_model.LinearRegression()
    y_regr.fit(x2.to_numpy(), y2.to_numpy())
    return [x_regr, y_regr]

In [6]:
def calibrate_files(taskname, output_subdirectory):
    calibration_df = create_calibration_df(taskname)
    [x_coeff, y_coeff] = calculate_regression_coefficients(calibration_df)
    for file in fileList:
        if '.csv' in file:
            df = pd.read_csv(data_directory + subdirectory + file)
            df = clean_df(df)
            df['CalibratedGazeAngleX'] = df.apply(lambda row: x_coeff.predict([[row['GazeAngleX'],row['GazeAngleY']]])[0], axis = 1)
            df['CalibratedGazeAngleY'] = df.apply(lambda row: y_coeff.predict([[row['GazeAngleX'],row['GazeAngleY']]])[0], axis = 1)
            if not os.path.exists(os.path.join(data_directory, output_subdirectory)):
                os.mkdir(os.path.join(data_directory, output_subdirectory))
            filePrefix = file.split('.')[0]
            #recalibrated_df_output_path = data_directory + output_subdirectory + '/' + filePrefix + '_recalibrated_' + taskname + '.csv'
            recalibrated_df_output_path = data_directory + output_subdirectory + '/' + file
            df.to_csv(recalibrated_df_output_path)

In [11]:
calibrate_files("calibration", "recalibrated_calibration")